In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
import numpy as np
from utils.dense_image_warp_functionnal import dense_image_warp_intermediate
from utils.depth_operations_functionnal import \
    get_disparity_sweeping_cv_former_intermediate, \
    get_disparity_sweeping_cv_former, tile_not_in_batch, get_coords_2d, \
    get_rot_mat, repeat_const
import tensorflow as tf
from tensorflow import keras as ks, float32
from keras import backend as K


@tf.function
def get_disparity_sweeping_cv(c1, c2, disp_prev_t, disp, rot, trans, camera,
                              search_range, nbre_cuts=1):
    """ Computes the DSCV as presented in the paper """

    # Prepare inputs
    nbre_copies = 2 * search_range + 1
    print("nbre_copies",nbre_copies)
    range_before_reshape = tf.range(-search_range, search_range + 1, 1.0, dtype=tf.float32)
    print("range_before_reshape", range_before_reshape.shape)
    expl_range = tf.reshape(range_before_reshape , [1, -1, 1, 1, 1])
    print("expl_range", expl_range.shape)
    b, h, w = c1.get_shape().as_list()[0:3]

    disp = tile_not_in_batch(disp, nbre_copies)
    print("disp_tile", disp.shape)
    disp = tf.reshape(disp, [-1, nbre_copies, w, h, 1])
    print("disp_reshape", disp.shape)

    print("expl_range", expl_range.shape)
    disp = disp + expl_range
    print("disp_sum=disp_sum_reshape=disp_reshape", disp.shape)
    disp = tf.clip_by_value(disp, 1e-6, 1e6)
    print("disp_clip", disp.shape)
    # Compute disp independent factors
    coords2d, _ = get_coords_2d(c1, camera)
    print("coords2d", coords2d.shape)
    coords2d = ks.layers.Reshape([h * w, 3, 1,])(coords2d)
    print("coords2d reshape", coords2d.shape)
    # rot_mat = tf.expand_dims(get_rot_mat(rot), axis=1)
    rot_mat = get_rot_mat(rot)
    print(rot_mat)
    rot_mat = ks.layers.Reshape((1, rot_mat.shape[1], rot_mat.shape[2],), )(rot_mat)
    print("rot mat", rot_mat)
    print(trans)
    t = ks.layers.Reshape((1, 3, 1, ), )(trans)

    myconst = tf.convert_to_tensor(np.ones((1, 1)).astype('float32'))
    ones_ = tf.keras.layers.Lambda(lambda x: repeat_const(x, myconst))(camera["f"])
    f_vec = ks.layers.Concatenate(axis=1)([camera["f"], ones_])
    f_vec =  ks.layers.Reshape((1, 3, 1,),)(f_vec)

    rot_coords = rot_mat @ coords2d
    alpha = rot_coords[:, :, -1:, :]
    proj_coords = rot_coords * f_vec / alpha
    scaled_t = t * f_vec
    print("ok")
    delta_x = scaled_t[:, :, 0, 0] - scaled_t[:, :, 2, 0] * proj_coords[:,
                                                            :, 0, 0]
    delta_y = scaled_t[:, :, 1, 0] - scaled_t[:, :, 2, 0] * proj_coords[:,
                                                            :, 1, 0]
    delta_x = ks.layers.Reshape([1, h, w, 1,],)(delta_x)
    delta_y = ks.layers.Reshape([1, h, w, 1,],)(delta_y)

    start_coords = ks.layers.Reshape([ 1, h, w, 2,],)(coords2d[:, :, :2, :] * f_vec[:, :, :2, :])
    proj_coords = ks.layers.Reshape([1, h, w, 2,],)(proj_coords[:, :, :2, :])
    print("proj_coords", proj_coords.shape)

    # disp to flow
    print("disp_to_flow", disp.shape)
    sqrt_value = tf.sqrt(delta_x ** 2 + delta_y ** 2)
    divider = sqrt_value / disp  # is correct computation after simplification

    delta = tf.concat([delta_x / divider, delta_y / divider], axis=-1)
    flow = proj_coords + delta - start_coords
    flow = tf.reverse(flow, axis=[-1])

    c1 = tile_not_in_batch(c1, nbre_copies)
    combined_data = tile_not_in_batch(tf.concat([c2, disp_prev_t], axis=-1),
                                  nbre_copies)

    print("flow2", flow.shape)
    print("c1", c1.shape)
    print("combined_data", combined_data.shape)

    combined_data_w = dense_image_warp_intermediate(combined_data, flow)
    # combined_data_w = combined_data

    print("combined_data_w", combined_data_w.shape)
    c2_w = combined_data_w[..., :-1]
    prev_disp = combined_data_w[..., -1]
    print("c2_w", c2_w.shape)
    print("prev_disp", prev_disp.shape)
    # Compute costs (operations performed in float16 for speedup)
    mult = tf.cast(c1, tf.float16) * tf.cast(c2_w, tf.float16)
    print("mult", mult.shape)
    split = tf.split(mult, num_or_size_splits=nbre_cuts, axis=-1)
    print("split", split)
    sub_costs = tf.stack(split, 1)
    print("sub_costs", sub_costs.shape)
    cv = tf.reduce_mean(sub_costs, axis=-1)
    print("cv", cv.shape)
    cv_to_cast = tf.reshape(cv, [-1, (nbre_cuts) * nbre_copies, h, w])
    print("cv_to_cast", cv_to_cast.shape)

    # cv_to_cast (18, 5, 48, 48)
    # cv_cast (5, 48, 48, 18)

    cv = tf.cast(tf.transpose(cv_to_cast, perm=[0, 2, 3, 1]), tf.float32)
    print("cv_cast", cv.shape)
    print("prev_disp", prev_disp.shape)
    tmp = prev_disp
    prev_disp = tf.transpose(prev_disp , perm=[0, 2, 3, 1])
    print("prev_disp_transposed", prev_disp.shape)
    return cv, prev_disp

h= 48
w = 48

c1 = ks.Input(shape=(h,w, 64,), dtype=float32)  # data image
c2 = ks.Input(shape=(h,w, 64,), dtype=float32)  # data image
disp_prev_t = ks.Input(shape=(h,w, 1,), dtype=float32)  # data image
disp = ks.Input(shape=(h,w, 1,), dtype=float32)  # data image
camera = {"f": ks.Input(shape=(2,), dtype=float32),
          "c": ks.Input(shape=(2,), dtype=float32)}

rot_input = ks.Input(shape=(4), dtype=float32,
                     name="rot_input")  # data camera displacement
trans_input = ks.Input(shape=(3), dtype=float32,
                       name="trans_input")  # data camera displacement
search_range=4
nbre_cuts=2

get_disparity_sweeping_cv_function = lambda x: get_disparity_sweeping_cv(x[0], x[1], x[2], x[3], x[4], x[5], x[6], search_range, nbre_cuts)
depth_curr_l = ks.layers.Lambda(get_disparity_sweeping_cv_function)((c1, c2,disp_prev_t, disp, rot_input, trans_input,camera))

model_full = ks.Model(inputs=[c1, c2,disp_prev_t, disp, rot_input, trans_input,camera],
                      outputs=depth_curr_l)

#get_disparity_sweeping_cv_output (TensorShape([1, 48, 48, 18]), TensorShape([1, 48, 48, 9]))
b=5
c1_test_1 = tf.random.uniform([b] + c1.shape[1:])
c2_test_1 = tf.random.uniform([b] + c2.shape[1:])
disp_prev_t_test_1 = tf.random.uniform([b] + disp_prev_t.shape[1:])
disp_test_1 = tf.random.uniform([b] + disp.shape[1:])
camera_test_1 = {"f":  tf.random.uniform([b] + camera["f"].shape[1:]),
          "c":  tf.random.uniform([b] + camera["c"].shape[1:])}
rot_test_1 = tf.random.uniform([b] + rot_input.shape[1:])
trans_test_1 = tf.random.uniform([b] + trans_input.shape[1:])
print()
print("---")
out_put_test_1_former = get_disparity_sweeping_cv_former(c1_test_1, c2_test_1, disp_prev_t_test_1, disp_test_1, rot_test_1, trans_test_1, camera_test_1, search_range, nbre_cuts=nbre_cuts)
print("out_put_test_1_former", out_put_test_1_former[0].shape, out_put_test_1_former[1].shape)
print("---")

out_put_test_1_former_intermediate = get_disparity_sweeping_cv_former_intermediate(c1_test_1, c2_test_1, disp_prev_t_test_1, disp_test_1, rot_test_1, trans_test_1, camera_test_1, search_range, nbre_cuts=nbre_cuts)
print("get_disparity_sweeping_cv_former_intermediate", out_put_test_1_former_intermediate[0].shape, out_put_test_1_former_intermediate[1].shape)
print("---")



out_put_test_1 = model_full((c1_test_1, c2_test_1, disp_prev_t_test_1, disp_test_1, rot_test_1, trans_test_1, camera_test_1))
print("get_disparity_sweeping_cv", out_put_test_1[0].shape, out_put_test_1[1].shape)
print("---")

# print(out_put_test_1_former[2][-1][0][0])
# print(out_put_test_1_former_intermediate[2][-1][0][0])
# print(tf.reduce_all(tf.math.equal(out_put_test_1_former_intermediate[2][0], out_put_test_1_former[2][0])))
print("---")
print("---")
print("---")
tensor1 = out_put_test_1_former[0]
tensor2 = out_put_test_1_former_intermediate[0]
tensor3 = out_put_test_1[0]
print(tensor1.shape, tensor2.shape,tensor3.shape)
# print(tensor1)
# print(tensor2)

print(tf.reduce_all(tf.math.equal(out_put_test_1_former_intermediate[0], out_put_test_1_former[0])))
print(tf.reduce_all(tf.math.equal(out_put_test_1_former_intermediate[0], out_put_test_1[0])))
print(tf.reduce_all(tf.math.equal(out_put_test_1_former_intermediate[1], out_put_test_1_former[1])))
print(tf.reduce_all(tf.math.equal(out_put_test_1_former_intermediate[1], out_put_test_1[1])))





nbre_copies 9
range_before_reshape (9,)
expl_range (1, 9, 1, 1, 1)
disp_tile (None, 9, 48, 48, 1)
disp_reshape (None, 9, 48, 48, 1)
expl_range (1, 9, 1, 1, 1)
disp_sum=disp_sum_reshape=disp_reshape (None, 9, 48, 48, 1)
disp_clip (None, 9, 48, 48, 1)
coords2d (None, 48, 48, 3, 1)
coords2d reshape (None, 2304, 3, 1)
Tensor("Reshape_2:0", shape=(None, 3, 3), dtype=float32)
rot mat Tensor("reshape_1/Reshape:0", shape=(None, 1, 3, 3), dtype=float32)
Tensor("trans:0", shape=(None, 3), dtype=float32)
ok
proj_coords (None, 1, 48, 48, 2)
disp_to_flow (None, 9, 48, 48, 1)
flow2 (None, 9, 48, 48, 2)
c1 (None, 9, 48, 48, 64)
combined_data (None, 9, 48, 48, 65)
functionnal
(1, 48, 48, 2)
(None, 9, 48, 48, 2)
image (None, 9, 48, 48, 65)
query_points_flattened (None, 9, 2304, 2)
num_queries Tensor("dense_image_warp/interpolate_bilinear/strided_slice_5:0", shape=(), dtype=int32) unstacked_query_points 2
batch_offsets (None, 9, 1)
flattened_grid (None, 20736, 65)
floors (None, 9, 2304)
linear_coordinat